# PDPRPy
## Praca Domowa nr 1
#### Sebastian Prokop
#### 2024-04-02 2

### Pobranie importów i wczytanie danych

In [1]:
# Pobranie potrzebnych importów
import pandas as pd
import os
import sqlite3

In [2]:
# Wczytanie danych
Posts = pd.read_csv("data/Posts.csv.gz", compression = 'gzip')
Comments = pd.read_csv("data/Comments.csv.gz", compression = 'gzip')
PostLinks = pd.read_csv("data/PostLinks.csv.gz", compression = 'gzip')
Users = pd.read_csv("data/Users.csv.gz", compression = 'gzip')

### Połączenie z bazą danych

In [3]:
# Każdą z ramek danych należy wyeksportować do bazy danych SQLite przy użyciu wywołania metody to_sql()
# w klasie pandas.DataFrame.

baza = os.path.join('stackexchangeDB.db')
if os.path.isfile(baza): # jesli baza już istneje
    os.remove(baza) # usuniemy ja zeby miec zaczac z czystą kartą

conn = sqlite3.connect(baza) 
Users.to_sql("Users", conn)
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)

121599

### Wczytaj wszystkie zapytania SQL

In [4]:
sql1 = '''SELECT Location, COUNT(*) AS Count
FROM (
SELECT Posts.OwnerUserId, Users.Id, Users.Location
FROM Users
JOIN Posts ON Users.Id = Posts.OwnerUserId
)
WHERE Location NOT IN ('')
GROUP BY Location
ORDER BY Count DESC
LIMIT 10
'''

sql2 = '''SELECT Posts.Title, RelatedTab.NumLinks
FROM
(
SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
FROM PostLinks
GROUP BY RelatedPostId
) AS RelatedTab
JOIN Posts ON RelatedTab.PostId=Posts.Id
WHERE Posts.PostTypeId=1
ORDER BY NumLinks DESC'''

sql3 = '''SELECT Title, CommentCount, ViewCount, CommentsTotalScore,
DisplayName, Reputation, Location
FROM (
SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount,
CmtTotScr.CommentsTotalScore
FROM (
SELECT PostId, SUM(Score) AS CommentsTotalScore
FROM Comments
GROUP BY PostId
) AS CmtTotScr
JOIN Posts ON Posts.Id = CmtTotScr.PostId
WHERE Posts.PostTypeId=1
) AS PostsBestComments
JOIN Users ON PostsBestComments.OwnerUserId = Users.Id
ORDER BY CommentsTotalScore DESC
LIMIT 10'''

sql4 = '''SELECT DisplayName, QuestionsNumber, AnswersNumber, Location,
Reputation, UpVotes, DownVotes
FROM (
SELECT *
FROM (
SELECT COUNT(*) as AnswersNumber, OwnerUserId
FROM Posts
WHERE PostTypeId = 2
GROUP BY OwnerUserId
) AS Answers
JOIN
(
SELECT COUNT(*) as QuestionsNumber, OwnerUserId
FROM Posts
WHERE PostTypeId = 1
GROUP BY OwnerUserId
) AS Questions
ON Answers.OwnerUserId = Questions.OwnerUserId
WHERE AnswersNumber > QuestionsNumber
ORDER BY AnswersNumber DESC
LIMIT 5
) AS PostsCounts
JOIN Users ON PostsCounts.OwnerUserId = Users.Id'''

sql5 = '''SELECT
Users.AccountId,
Users.DisplayName,
Users.Location,
AVG(PostAuth.AnswersCount) as AverageAnswersCount
FROM
(
SELECT
AnsCount.AnswersCount,
Posts.Id,
Posts.OwnerUserId
FROM (
SELECT Posts.ParentId, COUNT(*) AS AnswersCount
FROM Posts
WHERE Posts.PostTypeId = 2
GROUP BY Posts.ParentId
) AS AnsCount
JOIN Posts ON Posts.Id = AnsCount.ParentId
) AS PostAuth
JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
GROUP BY OwnerUserId
ORDER BY AverageAnswersCount DESC
LIMIT 10
'''

sql3_truncated = '''SELECT Title, CommentCount, ViewCount, CommentsTotalScore,
DisplayName, Reputation, Location
FROM (
SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount,
CmtTotScr.CommentsTotalScore
FROM (
SELECT PostId, SUM(Score) AS CommentsTotalScore
FROM Comments
GROUP BY PostId
) AS CmtTotScr
JOIN Posts ON Posts.Id = CmtTotScr.PostId
WHERE Posts.PostTypeId=1
) AS PostsBestComments
JOIN Users ON PostsBestComments.OwnerUserId = Users.Id
ORDER BY CommentsTotalScore DESC'''

sql2_part1 = '''
SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
FROM PostLinks
GROUP BY RelatedPostId
'''

sql2_part2 = '''SELECT Posts.Title, RelatedTab.NumLinks
FROM RelatedTab
JOIN Posts ON RelatedTab.PostId=Posts.Id
WHERE Posts.PostTypeId=1
ORDER BY NumLinks DESC'''


### Zrealizuj zapytania SQL

In [5]:
# Działa w mniej niż sekundę
ans1 = pd.read_sql_query(sql1, conn)
ans1

,Location,Count
0,"Christchurch, New Zealand",2795
1,"New York, NY",1872
2,"London, United Kingdom",1711
3,UK,1707
4,"Sunshine Coast QLD, Australia",1558
5,Australia,1199
6,Seat 21A,1056
7,"Vancouver, Canada",1051
8,Netherlands,1013
9,"London, UK",947


In [6]:
# Zostawiłem na 15 minut ale nadal się nie wykonało
# Żeby rozwiązać problem niekończącego się wywołania rozbijam 
# kwerendę na dwie części. W ten sposób wykonało się w 0.2s
ans2 = pd.read_sql_query(sql2_part1, conn)
ans2.to_sql(name = "RelatedTab", con = conn, index = False)
ans2 = pd.read_sql_query(sql2_part2, conn)
ans2

,Title,NumLinks
0,Is there a way to find out if I need a transit...,1157
1,Do I need a visa to transit (or layover) in th...,1012
2,Should I submit bank statements when applying ...,669
3,UK visa refusal on V 4.2 a + c (and sometimes ...,523
4,Should my first trip be to the country which i...,461
...,...,...
8094,Why can't I reserve a seat on the Eurocity tra...,1
8095,Independent Research in the US under B1 visa,1
8096,2 round tickets plus 1 back ticket to Thailand,1
8097,Which flight search engines show the flight op...,1


In [7]:
# Z zapytaniem 3 problem jest taki, że się nie kończy w podstawowej wersji.
# Żeby rozwiązać ten problem należy najpierw wykonać zapytanie z pominięciem
# ostatniej linii:
# LIMIT 10
# zapisać wynik do bazy danych, jako nową tabelę po czym z nowej tabeli 
# pobrać pierwsze 10 wierszy

ans3 = pd.read_sql_query(sql3_truncated, conn)
ans3.to_sql(name = "tempTable", con = conn, index = False) # Żeby nie dodawać kolumny nowej index=False
ans3 = pd.read_sql_query("SELECT * FROM tempTable LIMIT 10", conn)
ans3

,Title,CommentCount,ViewCount,CommentsTotalScore,DisplayName,Reputation,Location
0,"Boss is asking for passport, but it has a stam...",24,61309.0,630,Megha,1679,None
1,How to intentionally get denied entry to the U...,37,72856.0,618,user11743,1997,None
2,Can I wear a bulletproof vest while traveling ...,26,23467.0,403,Ulkoma,9190,"London, United Kingdom"
3,"OK we're all adults here, so really, how on ea...",29,113001.0,388,hippietrail,75994,"Sunshine Coast QLD, Australia"
4,Being separated from one's young children on a...,36,14308.0,358,Emma-louise,611,None
5,Immigration officer that stopped me at the air...,24,22495.0,345,DiegoJ,1273,Buenos Aires
6,How to avoid toddlers on a long-distance plane...,18,29344.0,343,Silver Dragon,943,"San Francisco, United States"
7,Can I fly with a gold bar?,25,55450.0,333,Ulkoma,9190,"London, United Kingdom"
8,How to book a flight if my passport doesn't st...,23,18029.0,330,kukis,894,"Sztokholm, Szwecja"
9,Why don't airlines have backup planes just in ...,26,18809.0,328,clickbait,542,None


In [8]:
# Działa bardzo szybko
ans4 = pd.read_sql_query(sql4, conn)
ans4

,DisplayName,QuestionsNumber,AnswersNumber,Location,Reputation,UpVotes,DownVotes
0,Mark Mayo,344,1968,"Christchurch, New Zealand",157193,16082,1661
1,phoog,12,1690,"New York, NY",120317,8767,567
2,Relaxed,8,1506,None,99185,11268,606
3,lambshaanxy,117,1466,None,98555,15021,275
4,Crazydre,172,1163,None,73180,2263,132


In [9]:
# Działa bardzo szybko
ans5 = pd.read_sql_query(sql5, conn)
ans5

,AccountId,DisplayName,Location,AverageAnswersCount
0,40811.0,vocaro,"San Jose, CA",11.0
1,280.0,csmba,"San Francisco, CA",11.0
2,44093.0,Emma Arbogast,"Salem, OR",10.0
3,204.0,Josh,Australia,10.0
4,11758.0,rvarcher,"Oklahoma City, OK",9.0
5,79346.0,Thomas Matthews,California,8.0
6,54571.0,Christian,None,8.0
7,42364.0,Petrogad,None,8.0
8,20473.0,Jeremy Boyd,"Houston, TX",8.0
9,19588.0,JD Isaacks,"Atlanta, GA",8.0


### Zamknięcie połączenia z bazą danych

In [10]:
conn.close()

## Realizacja zapytań za pomocą "zwykłych" metod i funkcji pakietu pandas

### Wgranie pliku z rozwiązaniami

In [11]:
from Sebastian_Prokop_PD2 import *

### Zadanie 1

In [12]:
# Rozwiązanie z pliku
sol1 = solution_1(Posts, Users)

# Porównanie wyników
ans1.equals(sol1)

True

### Zadanie 2

In [13]:
sol2 = solution_2(Posts, PostLinks)

# Żeby poradzić sobie z dziwnym sortowaniem z SQL - jeszcze jedno sortowanie wyniku
ans2 = ans2.sort_values(by = ['NumLinks', 'Title'], ascending = [False, True])
ans2.reset_index(drop = True, inplace = True)

ans2.equals(sol2)

True

### Zadanie 3

In [14]:
sol3 = solution_3(Comments, Posts, Users)
ans3.equals(sol3)

True

### Zadanie 4

In [15]:
sol4 = solution_4(Posts, Users)
ans4.equals(sol4) # radzi sobie z None i NaN

True

### Zadanie 5

In [16]:
sol5 = solution_5(Posts, Users)
ans5.equals(sol5)

True